### Parallel Quicksort (Java) [6 points]

Consider the following sequential algorithm for Quicksort, taken from Introduction to Algorithms by Cormen, Leiserson, Rivest, and Stein. Let `a: 0 .. N – 1 → integer` for constant `N ≥ 0` be a global variable. In algorithms, a call to procedure `p` with value parameters `E`, `F` and results `x`, `y` is written as `x, y ← p(E, F)`.

```algorithm
procedure partition(p, r: integer) → (q: integer)
    var x, i = a(r), p – 1
        for j = p to r – 1 do
            if a(j) ≤ x then
                i := i + 1 ; a(i), a(j) := a(j), a(i)
        a(i + 1), a(r) := a(r), a(i + 1)
        q := i + 1

procedure quicksort(p, r: integer)
    if p < r then
        q ← partition(p, r)
        quicksort(p, q – 1)
        quicksort(q + 1, r)
```

Implement Quicksort in Java by sorting in parallel after partitioning; for this, the parent thread can continue sorting one segment and a child thread is created for sorting the other segment. However, create a new thread only if both segments contain more than `S` elements; otherwise, sort sequentially both segments.

In [1]:
%%writefile Quicksort.java
import java.util.Random;

public class Quicksort {

    static int N;   // number of elements to be sorted
    static int S;   // threshold for creating a sub-thread
    static int a[]; // array to be sorted
    
    static int partition(int p, int r) {
        int x = a[r];
        int i = p - 1;
        for (int j = p; j <= r - 1; j++) {
            if (a[j] <= x) {
                i++; 
                int t = a[i]; a[i] = a[j]; a[j] = t;
            }
        }
        int t = a[i + 1]; a[i + 1] = a[r]; a[r] = t;
        return i + 1;
    }

    static void sequentialsort(int p, int r) {
        if (p < r) {
            int q = partition(p, r);
            sequentialsort(p, q - 1);
            sequentialsort(q + 1, r);
        }
    }

    static void parallelsort(int p, int r) {
        if (p < r) {
            int q = partition(p, r);

            if (q - p <= S || r - q <= S) {
                sequentialsort(p,q-1);
                sequentialsort(q+1,r);

            } else {
                Thread lt = new Thread(() -> parallelsort(p, q - 1));
                Thread rt = new Thread(() -> parallelsort(q+1,r));

                lt.start(); rt.start();

                try{
                    lt.join(); rt.join();
                } catch(Exception e){}
            }
        
        }
    }

    public static void main(String args[]) {
        N = Integer.parseInt(args[0]);
        S = Integer.parseInt(args[1]);
        a = new int[N];
        Random random = new Random();
        for (int i = 0; i < N; i++) a[i] = random.nextInt(10000);

        final long start = System.currentTimeMillis();
        parallelsort(0, N - 1);
        final long end = System.currentTimeMillis();
        
        for (int i = 1; i < N; i++) assert a[i - 1] <= a[i];
        System.out.println((end - start) + " ms");
    }
}

Overwriting Quicksort.java


Use the cell below to test your implementation.

In [2]:
!javac Quicksort.java
!java -enableassertions Quicksort 100000 10

1115 ms


Now observe the execution time for sorting `2000000` elements with various thresholds `S`. Add cells to record your observations. Run the program multiple times and keep the cell with the shortest elapsed time.

In [3]:
!java -enableassertions Quicksort 2000000 1

3318 ms


In [4]:
!java -enableassertions Quicksort 2000000 10

3860 ms


In [5]:
!java -enableassertions Quicksort 2000000 100

3998 ms


In [6]:
!java -enableassertions Quicksort 2000000 1000

229 ms


In [7]:
!java -enableassertions Quicksort 2000000 10000

95 ms


In [8]:
!java -enableassertions Quicksort 2000000 100000

227 ms


In [9]:
!java -enableassertions Quicksort 2000000 1000000

364 ms


Summarize your observations about the running time for varying values of `S` and explain!

for values of S from 1 - 100 the algorithm was noticeably slower with times ranging in the values of ~1800ms. with values of S from 1000 to 1000000 the algorithm was much more optimal with a consistent runtime of less then 400ms. Furthermore values of S from 10000 to 100000 the runtime was consistently faster than 200ms but when we change the S to 1000000 the run time gets a little slower with an average runtime of ~350ms